# 配当利回り

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/J-Quants/jquants-api-client-python/blob/master/examples/20220825-003-dividend.ipynb)


このノートブックでは J-Quants APIから取得できるデータから配当性向、配当利回り、および直近1,3ヶ月のリターンを一覧で表示します。


----

**このノートブックはGoogle Driveを使用します。**

- Google Drive の以下のファイルにリフレッシュトークンが書き込まれていることを想定しています。
    - `MyDrive/drive_ws/secret/jquantsapi-key.txt`
- Google Drive の以下のフォルダーにデータを書き込みます。
    - `MyDrive/drive_ws/marketdata`

In [ ]:
# 必要なモジュールをインストールします。
! python -m pip install jquants-api-client japanize-matplotlib

In [ ]:
# Google drive をマウントします。
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
from datetime import datetime

import japanize_matplotlib
import jquantsapi
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
# pandas の表示制限を調整します
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.width", 2000)

In [ ]:
# プロット用の設定をします
sns.set(rc={'figure.figsize': (15, 10)})
sns.set(font_scale=1.5)
sns.set_style('whitegrid')

japanize_matplotlib.japanize()

In [ ]:
# 一度取得したデータは Google Drive 上に保存して再利用します。
# 保存先ディレクトリを指定します。
STORAGE_DIR_PATH = "/content/drive/MyDrive/drive_ws/marketdata"
os.makedirs(STORAGE_DIR_PATH, exist_ok=True)
STORAGE_DIR_PATH

In [ ]:
# J-Quants APIのトークンを保存してあるファイルを指定します
REFRESH_TOKEN_FILE_PATH = "/content/drive/MyDrive/drive_ws/secret/jquantsapi-key.txt"

In [ ]:
def get_refresh_token(refresh_token_file_path: str = REFRESH_TOKEN_FILE_PATH):
    with open(refresh_token_file_path, "r") as f:
        refresh_token = f.read()
    return refresh_token.rstrip().lstrip()

In [ ]:
# ファイルからリフレッシュトークンを読み込みます
refresh_token = get_refresh_token()

In [ ]:
# J-Quants APIクライアントを初期化します
jqapi = jquantsapi.Client(refresh_token=refresh_token)

In [ ]:
# 銘柄情報を取得します
now = pd.Timestamp.now(tz="Asia/Tokyo")

list_file = f"{STORAGE_DIR_PATH}/list_{now.strftime('%Y%m%d')}.csv.gz"
if not os.path.isfile(list_file):
    df_list = jqapi.get_list()
    df_list.to_csv(list_file, compression="gzip", index=False)
    print(f"save file: {list_file}")

# ファイルからデータを読み込みます
print(f"file exists: {list_file}, loading")
df_list = pd.read_csv(list_file, dtype="str")

In [ ]:
# 株価情報を取得します
now = pd.Timestamp.now(tz="Asia/Tokyo")
# 過去3ヶ月のデータを取得
start_dt = now - pd.Timedelta(95, unit="D")
end_dt = now
if end_dt.hour < 18:
    # データ更新時間前の場合は日付を1日ずらします。
    end_dt -= pd.Timedelta(1, unit="D")
price_file = f"{STORAGE_DIR_PATH}/price_{start_dt.strftime('%Y%m%d')}_{end_dt.strftime('%Y%m%d')}.csv.gz"
if not os.path.isfile(price_file):
    df_p = jqapi.get_price_range(start_dt=start_dt, end_dt=end_dt)
    df_p.to_csv(price_file, compression="gzip", index=False)
    df_p.reset_index(drop=True, inplace=True)
    print(f"save file: {price_file}")

# データを読み込みます
print(f"file exists: {price_file}, loading")
df_p = pd.read_csv(price_file, dtype="str")
df_p.reset_index(drop=True, inplace=True)
# 各列のデータ型を調整します
df_p.loc[:, "Date"] = pd.to_datetime(df_p["Date"], format="%Y-%m-%d")
df_p.loc[:, "Open"] = df_p["Open"].astype(np.float64)
df_p.loc[:, "High"] = df_p["High"].astype(np.float64)
df_p.loc[:, "Low"] = df_p["Low"].astype(np.float64)
df_p.loc[:, "Close"] = df_p["Close"].astype(np.float64)
df_p.loc[:, "Volume"] = df_p["Volume"].astype(np.float64)
df_p.loc[:, "TurnoverValue"] = df_p["TurnoverValue"].astype(np.float64)
df_p.loc[:, "AdjustmentFactor"] = df_p["AdjustmentFactor"].astype(np.float64)
df_p.loc[:, "AdjustmentOpen"] = df_p["AdjustmentOpen"].astype(np.float64)
df_p.loc[:, "AdjustmentHigh"] = df_p["AdjustmentHigh"].astype(np.float64)
df_p.loc[:, "AdjustmentLow"] = df_p["AdjustmentLow"].astype(np.float64)
df_p.loc[:, "AdjustmentClose"] = df_p["AdjustmentClose"].astype(np.float64)
df_p.loc[:, "AdjustmentVolume"] = df_p["AdjustmentVolume"].astype(np.float64)

In [ ]:
# 過去3ヶ月に発表された財務情報を取得します
now = pd.Timestamp.now(tz="Asia/Tokyo")
start_dt = now - pd.Timedelta(90, unit="D")
end_dt = now
if end_dt.hour < 1:
    # データ更新時間前の場合は日付を1日ずらします。
    end_dt -= pd.Timedelta(1, unit="D")
statements_file = f"{STORAGE_DIR_PATH}/statements_{start_dt.strftime('%Y%m%d')}_{end_dt.strftime('%Y%m%d')}.csv.gz"
if not os.path.isfile(statements_file):
    cache_dir = f"{STORAGE_DIR_PATH}/raw_statements"
    os.makedirs(cache_dir, exist_ok=True)
    df_s = jqapi.get_statements_range(
        start_dt=start_dt, end_dt=end_dt, cache_dir=cache_dir
    )
    df_s.to_csv(statements_file, compression="gzip", index=False)
    print(f"save file: {statements_file}")

print(f"file exists: {statements_file}, loading")
df_s = pd.read_csv(statements_file, dtype="str")
# float64にするために"-"をnp.nanに置き換えます
df_s.replace({"－": np.nan}, inplace=True)
df_s["ResultDividendPerShareFiscalYearEnd"] = df_s["ResultDividendPerShareFiscalYearEnd"].astype(np.float64)
df_s["EarningsPerShare"] = df_s["EarningsPerShare"].astype(np.float64)
df_s["ForecastDividendPerShareAnnual"] = df_s["ForecastDividendPerShareAnnual"].astype(np.float64)
df_s["ForecastEarningsPerShare"] = df_s["ForecastEarningsPerShare"].astype(np.float64)
# 日付型に変換します
df_s.loc[:, "DisclosedDate"] = pd.to_datetime(
    df_s["DisclosedDate"], format="%Y-%m-%d"
)
df_s.loc[:, "CurrentPeriodEndDate"] = pd.to_datetime(
    df_s["CurrentPeriodEndDate"], format="%Y-%m-%d"
)
df_s.loc[:, "CurrentFiscalYearStartDate"] = pd.to_datetime(
    df_s["CurrentFiscalYearStartDate"], format="%Y-%m-%d"
)
df_s.loc[:, "CurrentFiscalYearEndDate"] = pd.to_datetime(
    df_s["CurrentFiscalYearEndDate"], format="%Y-%m-%d"
)
df_s.sort_values(["DisclosedDate", "DisclosedTime"], inplace=True)

In [ ]:
# 作業用にデータをコピーします
df_work = df_s.copy()
df_p_work = df_p.copy()

In [ ]:
# 財務情報を銘柄ごとに重複を排除して最新の財務情報のみを使用します
df_work.sort_values(["DisclosedDate", "DisclosedTime"], inplace=True)
df_work = df_work.drop_duplicates(["LocalCode"], keep="last")

# 終値が0の場合は前営業日の終値を使用します
df_p_work.sort_values(["Code", "Date"], inplace=True)
df_p_work["AdjustmentClose"].replace({0.0: np.nan}, inplace=True)
df_p_work.loc[:, "AdjustmentClose"] = df_p_work.groupby("Code")["AdjustmentClose"].ffill()
# 終値がnanの場合は翌営業日の終値を使用します (データの先頭)
df_p_work.loc[:, "AdjustmentClose"] = df_p_work.groupby("Code")["AdjustmentClose"].bfill()
# 各銘柄の直近のリターンを算出します
def _calc_return(df, bdays):
  return (df["AdjustmentClose"].iat[-1] / df["AdjustmentClose"].iloc[-bdays:].iat[0]) - 1
df_p_work.sort_values(["Code", "Date"], inplace=True)
df_returns_1months = df_p_work.groupby("Code").apply(_calc_return, 20).rename("1ヶ月リターン")
df_returns_3months = df_p_work.groupby("Code").apply(_calc_return, 60).rename("3ヶ月リターン")
# リターンと結合します
df_work = pd.merge(df_work, df_returns_1months, left_on=["LocalCode"], right_index=True, how="left")
df_work = pd.merge(df_work, df_returns_3months, left_on=["LocalCode"], right_index=True, how="left")

# 配当利回りを計算するために直近の終値を取得します
df_close = df_p_work.loc[df_p_work["Date"] == df_p_work["Date"].max(), ["Code", "Date", "AdjustmentClose"]]
# 直近の株価と結合します
df_work = pd.merge(df_work, df_close, left_on=["LocalCode"], right_on=["Code"], how="left")
# 配当利回りを算出します
df_work["配当利回り"] = df_work["ResultDividendPerShareFiscalYearEnd"] / df_work["AdjustmentClose"]
# 予想配当利回りを算出します
df_work["予想配当利回り"] = df_work["ForecastDividendPerShareAnnual"] / df_work["AdjustmentClose"]

# 配当性向を算出します
df_work["配当性向"] = df_work["ResultDividendPerShareFiscalYearEnd"] / df_work["EarningsPerShare"] 
# 予想配当性向を算出します
df_work["予想配当性向"] = df_work["ForecastDividendPerShareAnnual"] / df_work["ForecastEarningsPerShare"]

# 銘柄名と結合します
df_work = pd.merge(df_work, df_list, left_on=["LocalCode"], right_on=["Code"])

# 表示用に開示日を追加します
df_work["開示日"] = df_work["DisclosedDate"].dt.strftime("%Y-%m-%d")

In [ ]:
# 表示する項目を指定します
output_cols = [
    "LocalCode",
    "CompanyName",
    "開示日",
    "配当性向",
    "予想配当性向",
    "配当利回り",
    "予想配当利回り",
    "1ヶ月リターン",
    "3ヶ月リターン",
]
# 項目別に表示方法を指定します
output_format = {
    "配当性向": "{:.2%}",
    "予想配当性向": "{:.2%}",
    "配当利回り": "{:.2%}",
    "予想配当利回り": "{:.2%}",
    "1ヶ月リターン": "{:.2%}",
    "3ヶ月リターン": "{:.2%}",
}

In [ ]:
# 配当利回り順に上位20銘柄を表示します
df_work.sort_values(["配当利回り"], ascending=False)[output_cols].head(20).style.format(output_format)

In [ ]:
# 予想配当利回り順に上位20銘柄を表示します
df_work.sort_values(["予想配当利回り"], ascending=False)[output_cols].head(20).style.format(output_format)

In [ ]:
# 横軸に配当利回り、縦軸に1,3ヶ月リターンとして関係性をプロットします
display("1ヶ月リターン")
sns.jointplot(x='配当利回り', y='1ヶ月リターン', data=df_work.loc[df_work["配当利回り"] > 0.00], kind="reg")
plt.show()

display("３ヶ月リターン")
sns.jointplot(x='配当利回り', y='3ヶ月リターン', data=df_work.loc[df_work["配当利回り"] > 0.00], kind="reg")
plt.show()

In [ ]:
# 横軸に予想配当利回り、縦軸に1,3,6ヶ月リターンとして関係性をプロットします
display("1ヶ月リターン")
sns.jointplot(x='予想配当利回り', y='1ヶ月リターン', data=df_work.loc[df_work["予想配当利回り"] > 0.00], kind="reg")
plt.show()

display("３ヶ月リターン")
sns.jointplot(x='予想配当利回り', y='3ヶ月リターン', data=df_work.loc[df_work["予想配当利回り"] > 0.00], kind="reg")
plt.show()

In [ ]:
# 予想配当利回りが5%以上の銘柄とリターンの関係性をプロットします
display("1ヶ月リターン")
sns.jointplot(x='予想配当利回り', y='1ヶ月リターン', data=df_work.loc[df_work["予想配当利回り"] >= 0.05], kind="reg")
plt.show()

display("３ヶ月リターン")
sns.jointplot(x='予想配当利回り', y='3ヶ月リターン', data=df_work.loc[df_work["予想配当利回り"] >= 0.05], kind="reg")
plt.show()